# Python 指南

参考：[Python 指南](https://tvm.apache.org/ffi/guides/python_guide.html#)

在高层次上，tvm_ffi Python 包提供了一流的 Python 支持

- 表示 TVM FFI 中值的 Python 类 任何 ABI。
- 调用 TVM FFI ABI 兼容函数的机制。
- Python 值和 tvm_ffi 值之间的转换。

## Tensor

tvm_ffi 提供了托管的 DLPack 兼容 Tensor。

In [1]:
import os
# 保证 jupyter 中 nvcc 可以被找到并添加 CUDA 头文件路径
os.environ['PATH'] += ':/usr/local/cuda/bin'
# 添加 CUDA 头文件路径到 CPATH 环境变量，确保编译器能找到 cuda_runtime_api.h
if 'CPATH' in os.environ:
    os.environ['CPATH'] += ':/usr/local/cuda/include'
else:
    os.environ['CPATH'] = '/usr/local/cuda/include'

In [2]:
import numpy as np
import tvm_ffi

# 演示 NumPy 和 TVM FFI 之间的 DLPack 转换
np_data = np.array([1, 2, 3, 4], dtype=np.float32)
tvm_array = tvm_ffi.from_dlpack(np_data)
# 转换回 NumPy
np_result = np.from_dlpack(tvm_array)
# 验证结果是否与原始数据相等
np.testing.assert_array_equal(np_result, np_data)

在大多数情况下，不必显式创建张量。Python 接口可以接受 {class}`torch.Tensor` 和 {class}`numpy.ndarray` 对象并自动将它们转换为 {class}`tvm_ffi.Tensor`。


## 函数和回调

{class}`tvm_ffi.Function` 为 C++ 中的 `ffi::Function` 提供了 Python 接口。您可以通过 {func}`tvm_ffi.get_global_func` 检索全局注册的函数

In [3]:
import tvm_ffi

# testing.echo 是在 C++ 中定义和注册的函数，
# 其实现是简单的 lambda 表达式 [](ffi::Any x) { return x; }，该函数接收参数并原样返回。
fecho = tvm_ffi.get_global_func("testing.echo")
assert fecho(1) == 1

可以将 Python 函数作为参数传递给另一个 FFI 函数作为回调。在后台，调用 {func}`tvm_ffi.convert` 将 Python 函数转换为 {class}`tvm_ffi.Function`。

In [4]:
import tvm_ffi

# testing.apply 是在 C++ 中注册的函数
# [](ffi::Function f, ffi::Any val) { return f(x); }
fapply = tvm_ffi.get_global_func("testing.apply")
# 调用 fapply 并传入 lambda 回调函数作为 f
assert fapply(lambda x: x + 1, 1) == 2

这是非常强大的模式，允许将 Python 回调注入 C++ 代码中。您还可以将 Python 回调注册为全局函数。

In [5]:
import tvm_ffi

@tvm_ffi.register_global_func("example.add_one")
def add_one(a):
    return a + 1

assert tvm_ffi.get_global_func("example.add_one")(1) == 2

## 容器类型

当 FFI 函数从列表/元组中获取参数时，它们将被转换为 {class}`tvm_ffi.Array`。


In [6]:
import tvm_ffi

# Lists 变成 Arrays
arr = tvm_ffi.convert([1, 2, 3, 4])
assert isinstance(arr, tvm_ffi.Array)
assert len(arr) == 4
assert arr[0] == 1

字典将转换为 {class}`tvm_ffi.Map`

In [7]:
import tvm_ffi

map_obj = tvm_ffi.convert({"a": 1, "b": 2})
assert isinstance(map_obj, tvm_ffi.Map)
assert len(map_obj) == 2
assert map_obj["a"] == 1
assert map_obj["b"] == 2

当从 FFI 函数返回容器值时，它们也分别存储在这些类型中。

## 内联模块

还可以加载内联模块 ，其中 C++/CUDA 代码直接嵌入到 Python 脚本中，然后动态编译。例如，可以定义简单的内核，该内核为数组的每个元素加一，如下所示：

In [ ]:
import torch
from tvm_ffi import Module
import tvm_ffi.cpp

# define the cpp source code
cpp_source = '''
     void add_one_cpu(tvm::ffi::TensorView x, tvm::ffi::TensorView y) {
       // 库函数的实现
       TVM_FFI_ICHECK(x->ndim == 1) << "x 必须是一维张量"; 
       DLDataType f32_dtype{kDLFloat, 32, 1};
       TVM_FFI_ICHECK(x->dtype == f32_dtype) << "x 必须是浮点张量";
       TVM_FFI_ICHECK(y->ndim == 1) << "y 必须是一维张量";
       TVM_FFI_ICHECK(y->dtype == f32_dtype) << "y 必须是浮点张量";
       TVM_FFI_ICHECK(x->shape[0] == y->shape[0]) << "x 和 y 必须具有相同的形状";
       for (int i = 0; i < x->shape[0]; ++i) {
         static_cast<float*>(y->data)[i] = static_cast<float*>(x->data)[i] + 1;
       }
     }
'''

# 编译 C++ 源代码并加载模块
mod: Module = tvm_ffi.cpp.load_inline(
    name='hello', cpp_sources=cpp_source, functions='add_one_cpu'
)

# 使用加载模块中的函数执行运算
x = torch.tensor([1, 2, 3, 4, 5], dtype=torch.float32)
y = torch.empty_like(x)
mod.add_one_cpu(x, y)
torch.testing.assert_close(x + 1, y)

上面的代码使用 Python 脚本定义了 C++ 函数 add_one_cpu，动态编译它，然后加载编译后的 {class}`tvm_ffi.Module` 对象，通过 {func}`tvm_ffi.cpp.load_inline` 进行。然后，您可以像往常一样从模块调用函数 `add_one_cpu`。

## 加载模块

还可以通过 {func}`tvm_ffi.cpp.build_inline` 构建内联模块而无需直接加载。该函数会返回已构建的共享库，您可以随后使用 {func}`tvm_ffi.load_module` 来加载它。

In [9]:
# compile the cpp source code and load the module
lib_path: str = tvm_ffi.cpp.build_inline(
    name='hello',
    cpp_sources=cpp_source,
    functions='add_one_cpu'
)

# load the module
mod: Module = tvm_ffi.load_module(lib_path)

# use the function from the loaded module to perform
x = torch.tensor([1, 2, 3, 4, 5], dtype=torch.float32)
y = torch.empty_like(x)
mod.add_one_cpu(x, y)
torch.testing.assert_close(x + 1, y)

## 错误处理

FFI 函数可能会引发错误。在这种情况下，Python 包会自动将错误转换为 Python 中相应的错误类型

In [10]:
import tvm_ffi

# defined in C++
# [](String kind, String msg) { throw Error(kind, msg, backtrace); }
test_raise_error = tvm_ffi.get_global_func("testing.test_raise_error")

test_raise_error("ValueError", "message")

ValueError: message

还可以通过 {func}`tvm_ffi.register_error` 函数注册额外的错误派发。

## 高级：注册自定义对象

对于高级用例，可能需要注册自定义对象。这可以通过 TVM-FFI API 中的反射注册表来实现。

以 tvm_ffi 包的测试模块中 C++ 代码 为例：

```cpp
#include <tvm/ffi/reflection/registry.h>

// Step 1: Define the object class (stores the actual data)
class TestIntPairObj : public tvm::ffi::Object {
public:
  int64_t a;
  int64_t b;

  TestIntPairObj() = default;
  TestIntPairObj(int64_t a, int64_t b) : a(a), b(b) {}

  // Required: declare type information
TVM_FFI_DECLARE_OBJECT_INFO_FINAL("testing.TestIntPair", TestIntPairObj, tvm::ffi::Object);
};

// Step 2: Define the reference wrapper (user-facing interface)
class TestIntPair : public tvm::ffi::ObjectRef {
public:
  // Constructor
  explicit TestIntPair(int64_t a, int64_t b) {
    data_ = tvm::ffi::make_object<TestIntPairObj>(a, b);
  }

  // Required: define object reference methods
  TVM_FFI_DEFINE_OBJECT_REF_METHODS_NULLABLE(TestIntPair, tvm::ffi::ObjectRef, TestIntPairObj);
};

TVM_FFI_STATIC_INIT_BLOCK() {
  namespace refl = tvm::ffi::reflection;
  // register the object into the system
  // register field accessors and a global static function `__create__` as ffi::Function
  refl::ObjectDef<TestIntPairObj>()
    .def_ro("a", &TestIntPairObj::a)
    .def_ro("b", &TestIntPairObj::b)
    .def_static("__create__", [](int64_t a, int64_t b) -> TestIntPair {
      return TestIntPair(a, b);
    });
}
```

然后，可以为库中的对象创建包装类，如下所示：

```python
import tvm_ffi

# Register the class
@tvm_ffi.register_object("testing.TestIntPair")
class TestIntPair(tvm_ffi.Object):
    def __init__(self, a, b):
        # This is a special method to call an FFI function whose return
        # value exactly initializes the object handle of the object
        self.__init_handle_by_constructor__(TestIntPair.__create__, a, b)

test_int_pair = TestIntPair(1, 2)
# We can access the fields by name
# The properties are populated by the reflection mechanism
assert test_int_pair.a == 1
assert test_int_pair.b == 2
```

在后台，利用反射注册表注册的信息为每个类生成高效的字段访问器和方法。

重要的是，当你有多重继承时，你需要在基类和子类上调用 {func}`tvm_ffi.register_object`。